# Python Inventory System - Part 3: Database Operations

This notebook covers the database operations for our Inventory Management System, focusing on CRUD (Create, Read, Update, Delete) operations using SQLModel.

## What We'll Cover

1. CRUD operations overview
2. Adding new products
3. Editing existing products
4. Deleting products
5. Listing products
6. Best practices for database operations

## 1. CRUD Operations Overview

CRUD stands for:
- **C**reate: Adding new records
- **R**ead: Retrieving existing records
- **U**pdate: Modifying existing records
- **D**elete: Removing records

Our inventory system implements all four operations:

| Function | CRUD Operation | Description |
|----------|----------------|-------------|
| `add_product()` | Create | Adds a new product to inventory |
| `edit_product()` | Update | Modifies an existing product |
| `delete_product()` | Delete | Removes a product from inventory |
| `list_products()` | Read | Retrieves all products |

Let's explore each operation in detail.

## 2. Adding New Products

The `add_product()` function creates and stores a new product:

In [ ]:
from typing import List, Optional
from datetime import datetime
from sqlmodel import Session, select
from models import Product

def add_product(name: str, description: str, price: float, stock_quantity: int, category: str) -> Product:
    """
    Add a new product to the database
    
    Args:
        name: Product name
        description: Product description
        price: Product price
        stock_quantity: Current stock quantity
        category: Product category
        
    Returns:
        Product: The created product
    """
    with Session(engine) as session:
        product = Product(
            name=name,
            description=description,
            price=price,
            stock_quantity=stock_quantity,
            category=category
        )
        session.add(product)
        session.commit()
        session.refresh(product)
        return product

# Example usage
print("Adding a new product...")
new_product = add_product(
    name="Wireless Headphones",
    description="Noise-cancelling Bluetooth headphones",
    price=199.99,
    stock_quantity=25,
    category="Electronics"
)
print(f"Added product: {new_product}")

### Key Points:

1. **Session Management**:
   - Uses a context manager (`with` block) for automatic session cleanup
   - Session is used for a single logical operation

2. **Object Creation**:
   ```python
   product = Product(name=name, description=description, ...)
   ```
   - Creates a new Product instance
   - Fields like `created_at` and `updated_at` are automatically set

3. **Database Operations**:
   - `session.add(product)`: Marks the object for insertion
   - `session.commit()`: Flushes changes to the database
   - `session.refresh(product)`: Updates the object with database-generated values (like ID)

4. **Return Value**:
   - Returns the fully populated Product object
   - Includes database-generated fields like ID and timestamps

## 3. Editing Existing Products

The `edit_product()` function updates product information:

In [ ]:
def edit_product(
    product_id: int,
    name: Optional[str] = None,
    description: Optional[str] = None,
    price: Optional[float] = None,
    stock_quantity: Optional[int] = None,
    category: Optional[str] = None
) -> bool:
    """
    Edit an existing product in the database
    
    Args:
        product_id: ID of the product to edit
        name: New product name (if provided)
        description: New product description (if provided)
        price: New product price (if provided)
        stock_quantity: New stock quantity (if provided)
        category: New product category (if provided)
        
    Returns:
        bool: True if product was updated, False if not found
    """
    with Session(engine) as session:
        statement = select(Product).where(Product.id == product_id)
        product = session.exec(statement).first()
        
        if not product:
            return False
        
        if name is not None:
            product.name = name
        if description is not None:
            product.description = description
        if price is not None:
            product.price = price
        if stock_quantity is not None:
            product.stock_quantity = stock_quantity
        if category is not None:
            product.category = category
            
        product.updated_at = datetime.now()
        session.add(product)
        session.commit()
        return True

# Example usage
print("\nEditing the product we just created...")
success = edit_product(
    product_id=new_product.id,
    price=179.99,  # New discounted price
    stock_quantity=20  # Updated stock after some sales
)
print(f"Update successful: {success}")
print(f"Updated product price: ${new_product.price:.2f}")
print(f"Updated stock quantity: {new_product.stock_quantity}")

### Key Features:

1. **Partial Updates**:
   - All parameters are optional (using `Optional` type hints)
   - Only updates fields that are provided

2. **Product Lookup**:
   ```python
   statement = select(Product).where(Product.id == product_id)
   product = session.exec(statement).first()
   ```
   - Uses SQLModel's query API to find the product by ID
   - Returns None if product doesn't exist

3. **Update Logic**:
   - Checks each field for None before updating
   - Automatically updates `updated_at` timestamp

4. **Return Value**:
   - Returns boolean indicating success/failure
   - Makes it clear to callers whether the operation succeeded

## 4. Deleting Products

The `delete_product()` function removes products from inventory:

In [ ]:
def delete_product(product_id: int) -> bool:
    """
    Delete a product from the database
    
    Args:
        product_id: ID of the product to delete
        
    Returns:
        bool: True if product was deleted, False if not found
    """
    with Session(engine) as session:
        statement = select(Product).where(Product.id == product_id)
        product = session.exec(statement).first()
        
        if not product:
            return False
        
        session.delete(product)
        session.commit()
        return True

# Example usage
print("\nDeleting the product...")
success = delete_product(new_product.id)
print(f"Delete successful: {success}")

# Verify deletion
with Session(engine) as session:
    deleted_product = session.get(Product, new_product.id)
    print(f"Product still exists: {deleted_product is not None}")

### Implementation Details:

1. **Lookup Pattern**:
   - Similar to `edit_product()`, first finds the product by ID
   - Returns False if product doesn't exist

2. **Deletion**:
   ```python
   session.delete(product)
   session.commit()
   ```
   - Marks the object for deletion
   - Commits the transaction

3. **Safety**:
   - Never raises an exception if product doesn't exist
   - Cleanly returns False in that case

4. **Transaction**:
   - The deletion happens within a transaction
   - Will be rolled back if something fails

## 5. Listing Products

The `list_products()` function retrieves all products:

In [ ]:
def list_products() -> List[Product]:
    """
    List all products in the database
    
    Returns:
        List[Product]: List of all products
    """
    with Session(engine) as session:
        statement = select(Product).order_by(Product.name)
        products = session.exec(statement).all()
        return products

# Example usage
print("\nAdding sample products...")
sample_products = [
    add_product("Laptop", "High-performance laptop", 999.99, 10, "Electronics"),
    add_product("Mouse", "Wireless mouse", 29.99, 50, "Electronics"),
    add_product("Notebook", "Hardcover notebook", 12.99, 100, "Stationery")
]

print("\nListing all products:")
all_products = list_products()
for product in all_products:
    print(f"- {product.name} (${product.price:.2f}), Stock: {product.stock_quantity}")

### Key Aspects:

1. **Query Construction**:
   ```python
   statement = select(Product).order_by(Product.name)
   ```
   - Creates a SELECT query for all products
   - Orders results by product name

2. **Execution**:
   ```python
   products = session.exec(statement).all()
   ```
   - `exec()` runs the query
   - `all()` retrieves all results as a list

3. **Return Value**:
   - Returns a list of Product objects
   - Empty list if no products exist

4. **Performance**:
   - For large inventories, consider adding pagination
   - Current implementation loads all products into memory

## 6. Best Practices for Database Operations

1. **Session Management**:
   - Always use sessions in context managers (`with` blocks)
   - Keep sessions short-lived

2. **Error Handling**:
   - Consider adding try/except blocks for database errors
   - In production, you might want to add retry logic

3. **Transactions**:
   - Group related operations in transactions
   - Use `session.commit()` explicitly when needed

4. **Query Efficiency**:
   - Only select columns you need
   - Use `where()` clauses to filter data in the database

5. **Type Safety**:
   - Use Python type hints consistently
   - Helps catch errors early

6. **Testing**:
   - Test with a separate test database
   - Consider using fixtures to set up test data

## Summary

In this notebook, we've explored:

1. **CRUD operations** - The four fundamental database operations
2. **Product management** - Adding, editing, deleting, and listing products
3. **SQLModel usage** - How to work with sessions and queries
4. **Best practices** - For robust database operations

These operations form the core functionality of our inventory management system. In the next notebooks, we'll see how to build a user interface on top of these operations.